In [1]:
# Setting up the environment.
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
# Load the data from JHU
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
df_d = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [3]:
df.tail(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20
3258,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,...,128,132,138,155,164,170,167,180,187,200
3259,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,...,14,14,14,16,16,16,16,16,16,16
3260,84070020,US,USA,840,NaN,Weber-Morgan,Utah,US,41.271160,-111.914512,...,173,173,184,188,194,197,207,211,214,216


In [4]:
df_d.tail(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20
3258,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,...,3,3,3,3,3,3,3,3,3,3
3259,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,...,0,0,0,0,0,0,0,0,0,0
3260,84070020,US,USA,840,NaN,Weber-Morgan,Utah,US,41.271160,-111.914512,...,2,2,2,2,2,2,3,3,3,3


In [5]:
df = df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2' ,'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'], axis=1)
df_d = df_d.drop(['Population', 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2' ,'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'], axis=1)

In [6]:
df.head(3)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,149,149,151,151,151,151,152,152,152,154
2,0,0,0,0,0,0,0,0,0,0,...,15,15,15,16,16,19,19,19,19,19


In [7]:
# this cell by Leaf Petersen
# Copy the data frame
dfdaily=df.copy()

# We are on the first iteration
first=True
# For every column
for column in df:
   # If this is the first iteration
   if first:
     # Save the first column
     prev = column
     # Copy over the first column unchanged
     dfdaily[column]=df[column]
     # All subsequent iterations are not the first
     first=False
     # Skip the rest of the loop
     continue
   # We are not the first iteration, so keep going
   # Set the new column contents to the difference of the
   # previous column and the current column
   dfdaily[column] = df[column]-df[prev]
   # The current column is the previous column for the next iteration
   prev = column
dfdaily.head(3)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,4,0,2,0,0,0,1,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,3,0,0,0,0


In [8]:
# this cell by Leaf Petersen
# Copy the data frame
df_d_daily=df_d.copy()

# We are on the first iteration
first=True
# For every column
for column in df_d:
   # If this is the first iteration
   if first:
     # Save the first column
     prev = column
     # Copy over the first column unchanged
     df_d_daily[column]=df_d[column]
     # All subsequent iterations are not the first
     first=False
     # Skip the rest of the loop
     continue
   # We are not the first iteration, so keep going
   # Set the new column contents to the difference of the
   # previous column and the current column
   df_d_daily[column] = df_d[column]-df_d[prev]
   # The current column is the previous column for the next iteration
   prev = column
df_d_daily.head(3)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# We want the sum of all States and Counties to get the total value of Confirmed in the US
dfdaily = dfdaily.sum().to_frame()
dfdaily = dfdaily.reset_index()
# We want two colums named Date and Confirmed
dfdaily = dfdaily.rename(columns={'index': 'Date', 0: 'Confirmed'})
dfdaily.tail(3)

,Date,Confirmed
112,5/13/20,20832
113,5/14/20,27368
114,5/15/20,25050


In [10]:
# We want the sum of all States and Counties to get the total value of Deaths in the US
df_d_daily = df_d_daily.sum().to_frame()
df_d_daily = df_d_daily.reset_index()
# We want two colums named Date and Deaths
df_d_daily = df_d_daily.rename(columns={'index': 'Date', 0: 'Deaths'})
df_d_daily.head(3)

,Date,Deaths
0,1/22/20,0
1,1/23/20,0
2,1/24/20,0


In [11]:
# Calculating rolling average for window of 7
dfdaily_avg = dfdaily.rolling(7, win_type='triang').mean().round()
# Replacing NaNs 
dfdaily_avg.iloc[0] = [1]
dfdaily_avg = dfdaily_avg.fillna(0)
dfdaily_avg = dfdaily_avg.reset_index()
dfdaily_avg = dfdaily_avg.rename(columns={'index': 'Date'})
dfdaily_avg.head(3)

,Date,Confirmed
0,0,1.0
1,1,0.0
2,2,0.0


In [12]:
dfdaily_avg.head(3)

,Date,Confirmed
0,0,1.0
1,1,0.0
2,2,0.0


In [13]:
dfdaily_avg.tail(3)

,Date,Confirmed
112,112,22330.0
113,113,21617.0
114,114,21872.0


In [14]:
import plotly.graph_objects as go

# con = us_df['positive']
# dea = us_df['death']

fig = go.Figure()

fig.add_trace(go.Bar(x = df_d_daily['Date'], y = df_d_daily['Deaths'],
              #hovertext='Deaths',
              name='Deaths ',
              #opacity=.5,
              textposition='outside',
              hovertemplate = 'Deaths: <b>%{y:,.0f}</b><extra></extra>',
              marker_color='#ff3333'     
                    )
             )

fig.add_trace(go.Bar(x = dfdaily['Date'], y = dfdaily['Confirmed'],
              #hovertext='Positive cases',
              name='Confirmed ',
              textposition='outside',
              hoverlabel=dict(bgcolor='#ff8000', bordercolor='white',
                font=dict(family='Arial', size=20, color='white')
                             ),
              hovertemplate = 'Confirmed: <b>%{y:,.0f}</b><extra></extra>',
              marker_color='#ffa64d'
                    )
             )

fig.update_layout(#title=dict(text='Daily COVID19 Cases in the US (data: JHU)', x=0.5),
                  #width=1200,
                  barmode='group',
                  paper_bgcolor='rgba(0,0,0,0)', 
                  plot_bgcolor='rgba(0,0,0,0)',
                  uniformtext_minsize=7, uniformtext_mode='hide',
                  #hovermode='x', #sets the bar chart to display both tooltips by default
                  hovermode="x unified", # shows all tooltips in one box and vertical line - easier to read
                  hoverlabel=dict(
                                bgcolor="white", 
                                font_size=16, 
                                font_family="Arial",
                                bordercolor='#b3b3b3'),
                  legend=dict(x=0, y=1),
                  #font=dict(size = 12),
                  dragmode=False, #disable panning so mobile apps can receive pinch to zoom events
                  yaxis = dict(
                      scaleanchor = "x",
                      scaleratio = 1,
                              )
                 )
                 
                     
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='#a6a6a6', color='#4d4d4d',
                #showgrid=True, gridwidth=1, gridcolor='#f2f2f2',
                fixedrange=True,
                #tickfont = dict(size = 10),
                tickmode='auto',
                nticks=12,
                spikethickness=0.4,
                spikedash='solid',
                )
              
fig.update_yaxes(showline=True, linewidth=1, linecolor='#a6a6a6', color='#4d4d4d',
                 showgrid=True, gridwidth=1, gridcolor='#f2f2f2',
                 fixedrange=True,
                 tickfont = dict(size = 14),
                )


fig.update_traces(texttemplate='%{y:,.2s}', textposition='outside')


    
fig.add_trace(go.Scatter(x = dfdaily['Date'], y = dfdaily_avg['Confirmed'],
                    mode='lines',
                    fill='tozeroy',
                    name='',
                    line=dict(color=' #e67300', width=2),
                    showlegend=False,
                    hoverinfo='skip'
                        )
             )


fig.show(displayModeBar=False)
    
import plotly.io as pio
pio.write_html(fig, file='Index.html', auto_open=True)